### Import Libraries

In [35]:
! pip install geopy

import pandas as pd
from bs4 import BeautifulSoup
import requests
import numpy as np
from geopy.geocoders import Nominatim 
from pandas.io.json import json_normalize  

from sklearn.cluster import KMeans

import matplotlib.cm as cm
import matplotlib.colors as colors

### Gather Internet Data

In [8]:
source = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(source.text, 'lxml')

### Store Table Data In a List

In [17]:
data = []
columns = []
table = soup.find(class_='wikitable')
for index, tr in enumerate(table.find_all('tr')):
    section = []
    for td in tr.find_all(['th','td']):
        section.append(td.text.rstrip())
    if (index == 0):
        columns = section
    else:
        data.append(section)

canada_df = pd.DataFrame(data = data,columns = columns)
canada_df.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


### Remove "Not Assigned" Buroughs

In [18]:
canada_df = canada_df[canada_df['Borough'] != 'Not assigned']
canada_df.head()

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


### Finalize Table 

In [19]:
canada_df["Neighbourhood"] = canada_df.groupby("Postal Code")["Neighbourhood"].transform(lambda neigh: ', '.join(neigh))
canada_df = canada_df.drop_duplicates()

if(canada_df.index.name != 'Postal Code'):
    canada_df = canada_df.set_index('Postal Code')

canada_df['Neighbourhood'].replace("Not assigned", canada_df["Borough"],inplace=True)
canada_df.head()

,Borough,Neighbourhood
Postal Code,,
M3A,North York,Parkwoods
M4A,North York,Victoria Village
M5A,Downtown Toronto,"Regent Park, Harbourfront"
M6A,North York,"Lawrence Manor, Lawrence Heights"
M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


### Print the Number of Rows

In [20]:
canada_df.shape

(103, 2)

# Beginning of Part Two

### Use the CSV File to Gather the Coordinates

In [44]:
df_geo_coor = pd.read_csv('https://cocl.us/Geospatial_data')
df_geo_coor.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


### Combine the Two Dataframes

In [45]:
df_toronto = pd.merge(canada_df, df_geo_coor, how='left', left_on = 'Postal Code', right_on = 'Postal Code')
df_toronto.drop("Postal Code", axis=1, inplace=True)
df_toronto.head()

,Borough,Neighbourhood,Latitude,Longitude
0,North York,Parkwoods,43.753259,-79.329656
1,North York,Victoria Village,43.725882,-79.315572
2,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
